In [1]:
import wget
import ssl
import os
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# Enter NCRB Individual PDF url
pdf_url = 'https://ncrb.gov.in/sites/default/files/adsi_reports_previous_year/Table-2.2_2019_0.pdf'
fileName = pdf_url[67:]
try:
    #Delete the file to redownload it
    os.remove(fileName)
except FileNotFoundError as e:
    #Ignore in case FileNotFound Error occurs
    pass
# Download report
fileName = wget.download(pdf_url,fileName)

In [3]:
import numpy as np
import pandas as pd
import camelot

In [4]:
fileName

'Table-2.2_2019_0.pdf'

In [5]:
tables = camelot.read_pdf(fileName,
                          flavor='stream',
                          table_areas=[
                              "67.33570274636511,739.7126343736392,510.7894022617125,100.55518386407714"
                              ],
                          columns=[
                              "93.3080452342488,211.62649434571892,286.65770597738293,355.91728594507276,450.1872697899839"
                              ],
                              row_tol=10,
                          split_text=True,
                          pages="all"
                         )

In [6]:
tables

<TableList n=2>

In [7]:
def printnAllTables(tables):
    for i in range(len(tables)):
        print("Page No: ",(i+1))
        print(tables[i].df)
        print()

In [8]:
# Incase we're intersted in seeing the original scraped tables.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

In [9]:
# Removes Garbage Rows that may have been detected Before the 'STATES' row
# AND
# Removes Garbage Rows that may have been detected after the 'TOTAL (ALL INDIA)' row
def clean(table):
    while(table.df.iloc[0,1] != 'STATES' and table.df.iloc[0,1] != 'CITIES'):
        table.df = table.df.iloc[1:]
    while(table.df.iloc[-1,1] != 'TOTAL (ALL INDIA)' and table.df.iloc[-1,1] != 'TOTAL (CITIES)'):
        table.df = table.df.iloc[:-1]
    # The Return line is unnecessary due to all changes being reflected in the original references
    return table

In [10]:
for each_table in tables:
    clean(each_table)

In [11]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

In [12]:
def set_Column(df, city):
    column_list = [
        ("Sl. No."),
        ("State/UT" if (not city) else "City"), 
        ('Number of Suicides'),
        ('Percentage Share in Total Suicides'),
        ('Projected Mid–Year Population(in Lakh)'),
        ('Rate of Suicides')
    ]
    df.columns = column_list

In [13]:
for i in range(len(tables)):
    city = i==1
    set_Column(df=tables[i].df, city=city)
    # tables[i].df.set_index(["Sl. No.","State/UT" if (not city) else "City"],inplace=True)

In [14]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

# OR Print a simgle one according to your choice
# tables[23].df

In [15]:
# Reset Index before saving file, for better formatting in RAW CSV
tables[0].df.to_csv("Incidence and Rate of Suicides – 2019 (StateUT-wise) .csv",index=False)
tables[1].df.to_csv("Incidence and Rate of Suicides – 2019 (City-wise) .csv",index=False)